In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
import torch.nn.functional as F
from pprint import pprint

In [3]:
# モデルとトークナイザの読み込み
model_name = "koheiduck/bert-japanese-finetuned-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [5]:
# 動作確認
cls = pipeline('sentiment-analysis',model=model,tokenizer=tokenizer)
print(cls("私はとっても幸せ"))
print(cls("吾輩は猫である"))
print(cls("私はとっても不幸"))

[{'label': 'POSITIVE', 'score': 0.9896088242530823}]
[{'label': 'NEUTRAL', 'score': 0.9275946021080017}]
[{'label': 'NEGATIVE', 'score': 0.9918387532234192}]


In [30]:
cls(X)

[{'label': 'POSITIVE', 'score': 0.9896088242530823},
 {'label': 'NEUTRAL', 'score': 0.9275946021080017},
 {'label': 'NEGATIVE', 'score': 0.9918387532234192}]

In [29]:
X = [
    "私はとっても幸せ",
    "吾輩は猫である",
    "私はとっても不幸"
]
# トークナイズ処理
tokenized_text = tokenizer(X , 
                           padding=True, 
                           truncation=True, return_tensors='pt')
# スコア取得
with torch.no_grad():
    outputs = model(**tokenized_text)
    predictions = F.softmax(outputs.logits, dim=1)
    prediction = torch.argmax(predictions, dim=1)

print('logits : \n', outputs.logits.numpy())
print('predictions : \n', predictions.numpy())
print('prediction : \n', 
       prediction.numpy(),
       [model.config.id2label[p] for p in prediction.numpy()])

logits : 
 [[-1.3570828 -2.2782886  3.53434  ]
 [ 2.1544192 -0.7660058 -1.5692033]
 [-1.9613895  3.547533  -1.9304129]]
predictions : 
 [[0.00743268 0.0029585  0.9896088 ]
 [0.9275947  0.05000735 0.02239802]
 [0.00401741 0.99183875 0.00414381]]
prediction : 
 [2 0 1] ['POSITIVE', 'NEUTRAL', 'NEGATIVE']
